In [1]:
from compute import Par, Var, Model, adda, addf, Expcomp, Evaluable
from inputresolver import idx_eqlist, mdf_order, resolve, getallvars, invert_edges, reassign
from representations import bipartite_repr, digraph_repr
from notationcompute import optsolver, solver, explicit

In [2]:
p = Par(1, 'km')

In [3]:
p.varunit

1 <Unit('kilometer')>

In [8]:
p2 = Par()

In [9]:
p2.varval

In [10]:
p+p2

p0 + p2

In [44]:
from IPython.display import Math
Math(rf'0.1d+20+0.5b')

<IPython.core.display.Math object>

In [2]:
import openmdao.api as om
import networkx as nx
import sympy as sp

In [47]:
m = Model()
a = Var('a')
e = Var('e')
p = Par('p', 0.1)
x = adda(m, 'x', 0.5*a)
b = adda(m, 'b', x+0.5)
d = Var('d')
c = adda(m, 'c', p*d+20+0.5*b)
adda(m, d, c-10)
addf(m, d+0.5*e)

"Add(Mul(Par('p'), Var('d')), Mul(Float('0.5', precision=53), Var('b')), Integer(20))"

In [4]:
eqs, eqv, dout, dins = m.data_structures()

### Let's try to reassign all

In [5]:
B, edges = bipartite_repr(eqv)

In [6]:
M = nx.bipartite.maximum_matching(B)

In [7]:
doutM = {key:M[key] for key in dout.keys()}

In [8]:
doutM = {0: x, 1: b, 2: d, 3: c, 4: e}

In [9]:
doutM, dout

({0: x, 1: b, 2: d, 3: c, 4: e}, {0: x, 1: b, 2: c, 3: d, 4: None})

In [10]:
mdf_order(eqv, dout)

[0, 1, (3, 2), 4]

### Let's try to avoid loop

In [11]:
vrs =getallvars(eqs)-{None}

In [12]:
newdout = resolve(eqs, vrs, invert_edges(edges), maxiter=2, not_input=None)
newdout = dict(newdout)

Using license file C:\Users\johan\gurobi.lic
Academic license - for non-commercial use only - expires 2022-09-25
C []
1 0.0 []
C []
2 0.0 []


In [13]:
newdout = {0: a, 1: x, 2: b, 3: c, 4: d}

In [14]:
newdout, dout

({0: a, 1: x, 2: b, 3: c, 4: d}, {0: x, 1: b, 2: c, 3: d, 4: None})

In [15]:
mdf_order(eqs, newdout)

[4, 3, 2, 1, 0]

In [16]:
eqsn = reassign(eqs, newdout)

In [17]:
eqsn

{0: (a, 2.0*x),
 1: (x, b - 0.5),
 2: (b, 2.0*c - 0.2*d - 40.0),
 3: (c, d + 10),
 4: (d, -0.5*e)}

### Execution workflow 1

In [18]:
#mapping = {'solver':solver, 'exp':explicit}

In [19]:
prob = om.Problem()
model = prob.model
groups = {0:model}

In [20]:
explicit(groups, eqsn, 0, d, 4)
explicit(groups, eqsn, 0, c, 3)
explicit(groups, eqsn, 0, b, 2)
explicit(groups, eqsn, 0, x, 1)
explicit(groups, eqsn, 0, a, 0)

In [21]:
f = Var('f')
#explicit(groups, {5: (None, a**2+d**2)}, 0, f, 5)

In [22]:
optsolver(groups, {5: (None, a**2+d**2+b**2)}, 0, 1, [e], [[],[],[5]])

In [23]:
prob.setup();

In [24]:
#om.n2(model)

In [25]:
eqsn

{0: (a, 2.0*x),
 1: (x, b - 0.5),
 2: (b, 2.0*c - 0.2*d - 40.0),
 3: (c, d + 10),
 4: (d, -0.5*e)}

In [26]:
#prob.set_val('c', 100)
prob.set_solver_print(level=1)
prob.driver = om.ScipyOptimizeDriver()
prob.run_driver();

Optimization terminated successfully    (Exit mode 0)
            Current function value: [121.17674419]
            Iterations: 2
            Function evaluations: 3
            Gradient evaluations: 2
Optimization Complete
-----------------------------------


In [27]:
model.list_inputs();

9 Input(s) in 'model'

varname  val           
-------  --------------
eq4
  e      [-21.34883721]
eq3
  d      [10.6744186]  
eq2
  d      [10.6744186]  
  c      [20.6744186]  
eq1
  b      [-0.78604651] 
eq0
  x      [-1.28604651] 
eq5
  b      [-0.78604651] 
  d      [10.6744186]  
  a      [-2.57209302] 




In [28]:
prob.set_val('e', -21.35)
prob.run_model()
prob.get_val('f05'), prob.get_val('e')

(array([121.17675]), array([-21.35]))

### Execution workflow 2

In [33]:
prob = om.Problem()
model = prob.model
groups = {0:model}

## TODO
Need to exchange the numbers with the expressions probably to make the interface cleaner and independent of passing in the eqs var

In [34]:
solver(groups, eqs, 0, 1, [a], [4])
explicit(groups, eqs, 1, x, 0)
explicit(groups, eqs, 1, b, 1)
solver(groups, eqs, 1, 2, [c,d], [2,3])
optsolver(groups, {5: (None, a**2+d**2+b**2)}, 0, 1, [e], [[],[],[5]])

In [35]:
prob.setup();

In [37]:
prob.set_solver_print(level=1)
prob.driver = om.ScipyOptimizeDriver()
prob.run_driver();


group1
+  
+  =============
+  group1.group2
+  =============
+  NL: Newton Converged in 1 iterations
+  
+  =============
+  group1.group2
+  =============
+  NL: Newton Converged in 0 iterations
NL: Newton Converged in 1 iterations

group1
+  
+  =============
+  group1.group2
+  =============
+  NL: Newton Converged in 0 iterations
NL: Newton Converged in 0 iterations

group1
+  
+  =============
+  group1.group2
+  =============
+  NL: Newton Converged in 0 iterations
+  
+  =============
+  group1.group2
+  =============
+  NL: Newton Converged in 0 iterations
NL: Newton Converged in 1 iterations

group1
+  
+  =============
+  group1.group2
+  =============
+  NL: Newton Converged in 0 iterations
+  
+  =============
+  group1.group2
+  =============
+  NL: Newton Converged in 0 iterations
NL: Newton Converged in 1 iterations
Optimization terminated successfully    (Exit mode 0)
            Current function value: [121.17674419]
            Iterations: 2
            Function eva

In [36]:
# prob.set_val('e', -21.35)
# prob.run_model()
# prob.get_val('f05')

In [453]:
eqs = {
    0: (a, x**2-2*x),
    1: (None, a+sp.sin(x)-1),
    2: (None, -x+2)
}

In [454]:
groups = {}

In [459]:
prob = om.Problem()
model = prob.model
groups[0] = model

In [460]:
solver(0, 1, [x], [1])
explicit(1, a, 0)
#optsolver(0, 1, [x], [[2],[],[1]])

In [461]:
# for fx, *args in order:
#     mapping[fx](*args)

In [462]:
prob.setup();

In [463]:
#prob.driver = om.ScipyOptimizeDriver()

In [464]:
#prob.run_driver();

In [465]:
prob.set_solver_print(level=1)
prob.run_model()


group1
NL: Newton Converged in 6 iterations


In [466]:
model.list_outputs();

1 Explicit Output(s) in 'model'

varname  val         
-------  ------------
group1
  eq0
    a    [0.11550053]


1 Implicit Output(s) in 'model'

varname  val         
-------  ------------
group1
  res1
    x    [2.05617258]




In [408]:
om.n2(prob)

In [ ]:
group = parent.add_subsystem('group{}'.format(counter), 
        om.Group(), promotes=['*'])
cmp = Impcomp(output_names=['a','x'], equations=eqs(3,4))
parent.add_subsystem("eq{}".format(eqn), cmp)
cmp = Expcomp(output_name='b', equations=eq(0))
parent.add_subsystem("eq{}".format(eqn), cmp)
group = parent.add_subsystem('group{}'.format(counter), 
        om.Group(), promotes=['*'])
cmp = Impcomp(output_names=['c','d'], equations=eqs(1,2))

In [ ]:
g = prob.add
bal = BalanceComp()

bal.add_balance('c', val=1.0)
bal.add_balance('d', val=1.0)
prob.model.add_subsystem(name='balance', subsys=bal, promotes=['*'])

for eqn in [2,3]:
    parent.add_subsystem("eq{}".format(eqn), Expcomp(
                            output_name='r{}'.format(eqn),
                            equation=equationcreator(right-left),
                            debug=debug), 
                            promotes=['*'])
prob.model.connect('exec.y', 'balance.lhs:x')
prob.model.linear_solver = DirectSolver()
prob.model.nonlinear_solver = NewtonSolver(solve_subsystems=False)
prob.setup()
prob.set_val('exec.x', 2)
prob.run_model()